In [2]:
import pandas as pd
import numpy as np
import logging
import importlib
import ETL.config as config
import ETL.extract_data as extract
import ETL.utils as utils

importlib.reload(config)
importlib.reload(utils)

logger = logging.getLogger(__name__)

In [3]:
df = extract.extract_data(config.FILTERED_TRANSACTIONS_FILE_LOCAL,'DATE',['date'])

In [39]:
df.drop_duplicates()
df.sample(10)
print(len(df))

3954066


In [66]:
df.sample(10)

,date
2150514,2018-07-16 13:03:00
2871556,2019-01-21 08:39:00
3841546,2019-10-02 12:07:00
1234387,2017-11-18 21:33:00
477742,2017-05-05 14:48:00
1811144,2018-04-18 14:21:00
1364958,2017-12-23 06:13:00
3950679,2019-10-31 07:26:00
1355196,2017-12-20 13:08:00
1618184,2018-02-27 07:44:00


In [58]:
df['date_key'] = df['date'].str.replace(r'[^A-Za-z0-9]','',regex=True).str[:8]

In [7]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954066 entries, 0 to 3954065
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   date    3954066 non-null  object
dtypes: object(1)
memory usage: 30.2+ MB


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year']

In [12]:
df['date'].dt.year.value_counts()

date
2017    1399308
2018    1394792
2019    1159966
Name: count, dtype: int64

In [ ]:
df['date'].dt.month_name().value_counts()

date
November     230180
December     237131
February     320509
April        344230
September    344752
June         345242
October      353234
May          354319
January      355568
March        355889
July         356339
August       356673
Name: count, dtype: int64

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['day_of_week'] = df['date'].dt.day_of_week
df['day_type'] = (df['date'].dt.weekday > 5).map({True: 'Weekend', False: 'Weekday'})

In [34]:
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [67]:
df.sample(10)

,date
3388284,2019-06-06 06:22:00
573362,2017-05-30 16:19:00
2671293,2018-11-29 18:04:00
3289573,2019-05-11 06:53:00
2711988,2018-12-10 12:13:00
3185512,2019-04-13 16:37:00
1746531,2018-04-01 15:32:00
1729547,2018-03-28 10:47:00
3131932,2019-03-30 14:55:00
176952,2017-02-16 09:14:00


In [4]:
def create_dim_date(df: pd.DataFrame, data_name: str) -> pd.DataFrame:
    '''
    Creates a date dimension table from a given DataFrame containing date-related data.

    Parameters:
    df : pd.DataFrame
        Input DataFrame that includes at least one column with datetime or date information.
    
    data_name : str
        A label or identifier used to name or tag the generated date dimension table.

    Returns:
    pd.DataFrame
        A transformed DataFrame structured as a date dimension, with additional time-related columns such as:
        - year
        - quarter
        - month
        - day
        - day of week
        - flags for holiday or weekend
        - custom hierarchy levels (if needed)
    '''
    bins   = [-1, 15, 30, 45, 59] #use it to create ['minute_block']
    labels = ['0015', '1630', '3145', '4659'] #use it to create ['minute_block']
    

    try:
        logger.info(f'Starting ETL {data_name} data')
        
        df = df.drop_duplicates()
        df['date'] = pd.to_datetime(df['date'], errors='raise')
        dt = df['date'].dt

        df['year'] = dt.year
        df['quarter'] = dt.quarter
        df['month'] = dt.month.astype(str).str.zfill(2)
        df['month_name'] = dt.month_name()
        df['day'] = dt.day.astype(str).str.zfill(2)
        df['day_name'] = dt.day_name()
        df['day_of_week'] = dt.day_of_week
        df['day_type'] = (dt.weekday > 5).map({True: 'Weekend', False: 'Weekday'})
        df['hour'] = dt.hour.astype(str).str.zfill(2)
        df['minute_block'] = pd.cut(pd.to_datetime(df['date']).dt.minute, bins=bins, labels=labels).astype(str)
        #df['date_key'] = df['date'].astype(str).str.replace(r'[^A-Za-z0-9]','',regex=True).str[:8] + df['minute_block'].astype(str)
        
        df['date_key'] = (
        df['year'].astype(str) +
        df['month'].astype(str) +
        df['day'].astype(str) +
        df['hour'].astype(str) +
        df['minute_block']
        )
        df['date_key'] = df['date_key'].astype(int)

        return df.drop(columns='date').drop_duplicates()

    except Exception as e:
        logger.info(f'Error in creating date dimension for {data_name}: {e}')
        raise

In [5]:
df.sample(5)

,date
2928548,2019-02-05 08:02:00
3050141,2019-03-09 08:52:00
2387329,2018-09-16 09:51:00
3482097,2019-06-30 12:13:00
3573751,2019-07-24 10:53:00


In [6]:
dim_date = create_dim_date(df,'dim_date')

C:\Users\jimmy\AppData\Local\Temp\ipykernel_28456\1166741491.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], errors='raise')
C:\Users\jimmy\AppData\Local\Temp\ipykernel_28456\1166741491.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = dt.year
C:\Users\jimmy\AppData\Local\Temp\ipykernel_28456\1166741491.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [7]:
dim_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99254 entries, 0 to 3954051
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          99254 non-null  int32 
 1   quarter       99254 non-null  int32 
 2   month         99254 non-null  object
 3   month_name    99254 non-null  object
 4   day           99254 non-null  object
 5   day_name      99254 non-null  object
 6   day_of_week   99254 non-null  int32 
 7   day_type      99254 non-null  object
 8   hour          99254 non-null  object
 9   minute_block  99254 non-null  object
 10  date_key      99254 non-null  int64 
dtypes: int32(3), int64(1), object(7)
memory usage: 8.0+ MB


In [202]:
dim_date.sample(10)

,year,quarter,month,month_name,day,day_name,day_of_week,day_type,hour,minute_block,date_key
217616,2017,1,02,February,26,Sunday,6,Weekend,17,4659,20170226174659
616252,2017,2,06,June,10,Saturday,5,Weekday,16,4659,20170610164659
728706,2017,3,07,July,10,Monday,0,Weekday,03,3145,20170710033145
1148130,2017,4,10,October,27,Friday,4,Weekday,13,0015,20171027130015
666139,2017,2,06,June,23,Friday,4,Weekday,16,4659,20170623164659
665519,2017,2,06,June,23,Friday,4,Weekday,14,1630,20170623141630
324558,2017,1,03,March,26,Sunday,6,Weekend,20,0015,20170326200015
3744625,2019,3,09,September,06,Friday,4,Weekday,20,3145,20190906203145
2527840,2018,4,10,October,23,Tuesday,1,Weekday,10,0015,20181023100015
271852,2017,1,03,March,13,Monday,0,Weekday,01,3145,20170313013145


In [182]:
df = df.drop_duplicates()
df.shape
df.sample(3)

1283060    2017-12-01 15:17:00
713301     2017-07-06 02:06:00
2129110    2018-07-10 23:07:00
Name: date, dtype: object

In [ ]:
dim_date.loc[dim_date['date_key'].astype(str).str.contains('^20180101')]

,year,quarter,month,month_name,day,day_name,day_of_week,day_type,hour,minute_block,date_key
1399308,2018,1,1,January,1,Monday,0,Weekday,0,015,2018010100015
1399318,2018,1,1,January,1,Monday,0,Weekday,0,1630,20180101001630
1399322,2018,1,1,January,1,Monday,0,Weekday,0,3145,20180101003145
1399333,2018,1,1,January,1,Monday,0,Weekday,0,4659,20180101004659
1399339,2018,1,1,January,1,Monday,0,Weekday,1,015,2018010101015
...,...,...,...,...,...,...,...,...,...,...,...
1403339,2018,1,1,January,1,Monday,0,Weekday,22,4659,20180101224659
1403367,2018,1,1,January,1,Monday,0,Weekday,23,015,2018010123015
1403375,2018,1,1,January,1,Monday,0,Weekday,23,1630,20180101231630
1403383,2018,1,1,January,1,Monday,0,Weekday,23,3145,20180101233145


In [122]:
dim_date.shape

(99254, 11)

In [ ]:
#alternative function
def create_dim_date(df: pd.DataFrame, data_name: str) -> pd.DataFrame:
    '''
    Creates a date dimension table from a given DataFrame containing date-related data.

    Parameters:
    df : pd.DataFrame
        Input DataFrame that includes at least one column with datetime or date information.
    
    data_name : str
        A label or identifier used to name or tag the generated date dimension table.

    Returns:
    pd.DataFrame
        A transformed DataFrame structured as a date dimension, with additional time-related columns such as:
        - year
        - quarter
        - month
        - day
        - day of week
        - flags for holiday or weekend
        - custom hierarchy levels (if needed)
    '''
    bins   = [-1, 15, 30, 45, 59] #use them to create ['minute_block']
    labels = ['015', '1630', '3145', '4659'] #use them to create ['minute_block']
    

    try:
        logger.info(f'Starting ETL {data_name} data')
        
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'])
        dt = df['date'].dt

        df['year'] = dt.year
        df['quarter'] = dt.quarter
        df['month'] = dt.month
        df['month_name'] = dt.month_name()
        df['day'] = dt.day
        df['day_name'] = dt.day_name()
        df['day_of_week'] = dt.day_of_week
        df['day_type'] = (dt.weekday > 5).map({True: 'Weekend', False: 'Weekday'})
        df['hour'] = dt.hour
        df['minute_block'] = pd.cut(pd.to_datetime(df['date']).dt.minute, bins=bins, labels=labels)
        df['date_key'] = df['date'].astype(str).str.replace(r'[^A-Za-z0-9]','',regex=True).str[:8] + df['minute_block'].astype(str)
        df['date_key'] = df['date_key'].astype(int)
        
        return df.drop(columns= 'date').drop_duplicates()

    except Exception as e:
        logger.info(f'Error in creating date dimension for {data_name}: {e}')
        raise

In [18]:
data = pd.Series(['$12345', '$67890'])
cleaned = data.replace(r'[/$]', '', regex=True).astype(int)
print(cleaned)


0    12345
1    67890
dtype: int64
